# Loading, Extracting and Cleaning Data

In [1]:
# Importing necessary libraries

import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')


In [2]:
# load file
with open('Intents_bigfile.json') as f:
    intents = json.load(f)

In [3]:
# preprocessing function
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [4]:
# extracting intent, text and response data after using the preprocessing function

inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

# Tokenizing input data and creating categorical targets

In [5]:
# Tokenization is a method to segregate a particular text into small chunks or tokens. 
# Here the tokens or chunks can be anything from words to characters, even subwords.
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

In [6]:
# tokenize input data
tokenizer, input_tensor = tokenize_data(inputs)

In [7]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

In [8]:
# categorical target output data

target_tensor, trg_index_word = create_categorical_target(targets)

In [9]:
# How does our input and output look?
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))


input shape: (143, 9) and output shape: (143, 22)


# Building model


In [10]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [11]:
# Recurrent Neural Network with Tensor flow

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         66048     
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                2838      
                                                                 
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
__________________________________________________

In [12]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)


In [13]:
# Training
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])


Epoch 1/50
5/5 [==============================] - 2s 22ms/step - loss: 3.0941 - accuracy: 0.0909
Epoch 2/50
5/5 [==============================] - 0s 19ms/step - loss: 2.6479 - accuracy: 0.2657
Epoch 3/50
5/5 [==============================] - 0s 19ms/step - loss: 1.8712 - accuracy: 0.4755
Epoch 4/50
5/5 [==============================] - 0s 24ms/step - loss: 1.2177 - accuracy: 0.6154
Epoch 5/50
5/5 [==============================] - 0s 23ms/step - loss: 0.6717 - accuracy: 0.7832
Epoch 6/50
5/5 [==============================] - 0s 23ms/step - loss: 0.3628 - accuracy: 0.9021
Epoch 7/50
5/5 [==============================] - 0s 22ms/step - loss: 0.1504 - accuracy: 0.9510
Epoch 8/50
5/5 [==============================] - 0s 21ms/step - loss: 0.1123 - accuracy: 0.9580
Epoch 9/50
5/5 [==============================] - 0s 21ms/step - loss: 0.0637 - accuracy: 0.9720
Epoch 10/50
5/5 [==============================] - 0s 22ms/step - loss: 0.0364 - accuracy: 0.9930
Epoch 11/50
5/5 [============

# The Chat Bot

In [14]:
# response function
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

In [15]:
# Bot function

print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} '.format(res))
    print()

Note: Enter 'quit' to break the loop.
You: Hello!
Bot: It's life Jim but not as we know it! 

You: What is your name?
Bot: You are <HUMAN>! How can I help? 

You: Can you tell me a joke?
Bot: A man goes into a bar and orders a pint. After a few minutes he hears a voice that says, 'Nice shoes'. He looks around but the whole bar is empty apart from the barman at the other end of the bar. A few minutes later he hears the voice again. This time it says, 'I like your shirt'. He beckons the barman over and tells him what's been happening to which the barman replies, 'Ah, that would be the nuts sir. They're complimentary'! 

You: Can you tell me the weather in Omaha?
Bot: Let me see 

You: quit
